# Контекстные менеджеры

https://pavel-karateev.gitbook.io/intermediate-python/sintaksis/context_managers

Иногда при работе с какими-то сущностями нужно корректно обработать их закрытие в любом случае, даже при возникновении исключения. Классический пример - любой открытый файловый дескриптор нужно закрыть, во избежание блокировки ресурса. Для удобства этого в Python 2.5 появились контекстные менеджеры.

In [1]:
with open("some_file.txt", 'w') as f:
    print(f)

<_io.TextIOWrapper name='some_file.txt' mode='w' encoding='cp1251'>


В предыдущем примере функция `open` возвращает объект контекстного менеджера `TextIOWrapper`. Интерфейс контекстного менеджера следующий:

- `__enter__(self)` - метод, который вызывается в начале работы с контекстным менеджером. Должен возвращать сам контекстный менеджер
- `__exit__(self, type, value, traceback)` - метод, в котором описываются действия в конце работы с контекстным менеджером

In [2]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method, encoding='utf-8')
        
    def __enter__(self):
        return self.file_obj
    
    def __exit__(self, type_, value, traceback):
        self.file_obj.close()
        
    def __del__(self):
        print("Сработал деструктор")
        

with File('demo.txt', 'w') as opened_file:
    opened_file.write('Привет!')

Сработал деструктор


Алгоритм работы контекстного менеджера

1. `with` сохраняет метод `__exit__` класса `File`.
2. Следует вызов метода `__enter__` класса `File`.
3. Метод `__enter__` открывает файл и возвращает его.
4. Дескриптор файла передается в `opened_file`.
5. Мы записываем информацию в файл при помощи `.write()`.
6. `with` вызывает сохраненный `__exit__` метод.
7. Метод `__exit__` закрывает файл.
8. Объект контекстного менеджера удаляется.

### Обработка исключений в контекстном менеджере

Иногда в зависимости от типа ошибки, которая произошла при выполнении кода внутри контекстного менеджера, нужно по-разному завершать работу с контекстным менеджером. Для этого как раз существуют параметры `type, value, traceback` метода `__exit__`

In [4]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        return self.file_obj
    
    def __exit__(self, type_, value, traceback):
        print("Исключение было обработано")
        self.file_obj.close()
        return True
    
    def __del__(self):
        print("Сработал деструктор")

    
with File('demo.txt', 'w') as opened_file:
    opened_file.undefined_function()

Исключение было обработано
Сработал деструктор


Алгоритм обработки исключений

1. Тип, значение и обратная трассировка ошибки передается в метод `__exit__`.
2. Обработка исключения передается методу `__exit__`
3. Если `__exit__` возвращает `True`, то исключение было корректно обработано.
4. При возврате любого другого значения `with` вызывает исключение.

## Contextlib

Есть еще более компактный способ создания контекстных менеджеров - через функцию-генератор.

In [5]:
from contextlib import contextmanager


@contextmanager
def open_file(name):
    # код, который мы бы записали в __enter__
    f = open(name, 'w')
    yield f
    # код, который мы бы записали в __exit__
    f.close()
    

with open_file('demo.txt') as f:
    f.write("contextlib")

## Задание

Написать контекстный менеджер `Timer`, который после выполнения кода внутри своего блока будет выводить на экран свое название, полученное в конструкторе, и количество секунд (целое число), которое этот блок выполняется. Если во время выполнения кода возникает исключение, на экран вместо секунд нужно вывести строку `EXCEPTION`. Кроме того, объект, передаваемый в блок контекстного менеджера должен быть вызываемым и по вызову возвращать текущее количество секунд, прошедшее с начала исполнения блока.

#### Пример использования:

<code>import time


with Timer("timer_name") as timer:
    time.sleep(2)
    print(timer())
    time.sleep(2)</code>

Вывод этого кода:

<code>2
timer_name 4</code>

#### Пример с вызовом исключения:

<code>with Timer("timer_name") as timer:
    1 / 0</code>
    
Вывод для этого кода:

<code>EXCEPTION</code>